### Etivity 4 - Tensors

#### Student Name:   Mark Murnane

#### Student ID:     18195326
___
**Problem Description:**

With the final version of the code from the peer you selected in E-tivity 3 as a starting point, create a class that contains one tensor and provides the following methods:

* Calculate the size of the tensor as a 2D tuple
* Sum/subtract the tensor with another tensor of suitable size
* Multiply the tensor with a tensor of suitable size
* Calculate the determinant of the tensor, but only for tensors of size 2x2 (matrix)
* Calculate the inverse of the tensor, but only for tensors of size 2x2
* Calculate the cross product of the tensor with another tensor of suitable size

___
**Guidelines**

* Write your code without the use of imported modules.
* Provide suitable exception handling to prevent the above operations from being used on unsuitable tensors.
* Test your code with one example for each of the following scenarios:
* sum of a 4x4 with a 4x4 matrix
* multiplication of a 4x4 with a 4x4 matrix
* multiplication of a 4x4 matrix with a suitable vector
* determinant of a 2x2 matrix
* inverse of a 2x2 matrix
* cross-product of 2 suitable tensors
___
**Source Base**
For E-tivity 3 I reviewed Andrew Kenny's code.  This original code can be viewed in GitLab [here](https://gitlab.com/CE4021/Group2/week3/blob/AndrewKenny/Etivity3_18203442.ipynb).

### Approach

In line with the Problem Description, I created a Tensor class and used Andrew's functions as the core methods for the class.

As Andrew's implementation was function based, I added class variables to hold the dimensions of the Tensor, as well as it's elements.  To facilitate class creation, I added an `__init__` method that takes either a list or tuple representation of the tensor, validates it and stores it internally.  As my own E-tivity 3 implementation ([here](https://gitlab.com/CE4021/Group2/week3/blob/MarkMurnane/Etivity3.ipynb)) used a class, I copied this method over.

#### Modifications to Andrew's code

Instead of using the class structure as a pseudo-module for tensor calculations on other structures, I decided to implement the Tensor class as an operable unit, i.e. that it is the subject of operations.   To use Andrew's functions as the methods of the class, the parameters had to be adjusted to use `self` as the first parameter.  This altered the signature of the (now) methods.

For the initial release, I've tried to minimise the internal changes, by renaming method parameters and then assigning them to method-scoped names, so that the code can be re-used without modification (e.g. `common_calculation`).  Some functions changed more radically by the fact that there are now object properties.  For example, as the size/shape of the Tensor is now a property of the class, the `get_size` method was reduced to simply returning the value of the property.

#### Incorporating my own Etivity 3 Code

To facilitate unit testing for the conversion to classes, I've taken the override operators I deinfed in my own Etivity 3 code and implemented them in this `Tensor`class.  Instead of calling out to my own code, they call out to Andrew's code.  

One additional change that was made here was to include my own validity checks for multiplication.  This was not present elsewhere.

The unit tests themselves were modified to check for ValueErrors as used by Andrew.

#### New functionality for Etivity 4

* Added a new method `det()` to calculate the determinant of the Tensor object.  Following the initial submission, method has been expanded to nxn matrices.  Specific cases are used for 1x1 and 2x2 matrices.  Larger matrices use the Laplace expansion.  This is not the most computationally efficient, but is relatively easy to follow.  The method returns a `ValueError` for non-square matrices.  


* Added a new method `inverse()` to calculate the inverse of a Tensor object.  This is currently confined to a 2x2 matrix.  This method works on the basis of:

$
\quad\quad\quad\quad
A = \begin{bmatrix}
    a & b \\
    c & d \\
\end{bmatrix}
\quad\quad\quad\quad
$
$
\quad\quad\quad\quad
A^{-1} = \frac{1}{\det A} \begin{bmatrix}
    d & -c \\
    -b & a \\
\end{bmatrix}
\quad\quad\quad\quad
$



* Added a new method `cross_product(other)` to calculate the cross product of the Tensor object and another Tensor object.  The formula calculates for 3x1 or 1x3 vectors.  While I use 3x1, some prefer 1x3.  While some material introduces the concept using 2D Vectors, these are not really considered valid so I've focused on 3D Vectors.  Higher dimensions 
are possible (based on my reading) but possibly much higher dimensions (7D).<br/><br/>
The formula used for 2 x 3D vectors $\hat{v}$ and $\hat{w}$ is:

$
\quad\quad\quad\quad
\vec{v} = \begin{bmatrix}
    v_x \\
    v_y \\
    v_z
\end{bmatrix}
\quad\quad\quad\quad
$
$
\quad\quad\quad\quad
\vec{w} = \begin{bmatrix}
    w_x \\
    w_y \\
    w_z
\end{bmatrix}
\quad\quad\quad\quad
$
$
\quad\quad\quad\quad
\vec{v} \times \vec{w} = \vec{p} = \begin{bmatrix}
    v_{y}w_{z} - v_{z}w_{y} \\
    v_{z}w_{x} - v_{x}w_{z} \\
    v_{x}w_{y} - v_{y}w_{x} \\
\end{bmatrix}
$

_Reflection_:

Although some of my peers have implemented cross product for 2D vectors, I've elected not to do this.  Transforming to a 3D matrix with a z-component = 0 would be straightforward (and is what numpy does).  Numpy returns the determinant of the matrix formed by those vectors, or assumes that the z-component is 0.  I'm not sure of the validity of either approach.  I feel it won't always be true, but perhaps I just don't understand the maths properly.

Pep has confirmed in the forums that 2D vectors don't need to be handled.  

In [1]:
# Student Name:    Mark Murnane
# Student ID:      18195326

class Tensor(object):
    
    """This class represents a mathematical Tensor for use in Linear Algebra
    
    In this initial implementation a tensor is confined to maximum of 2 dimensions, essentially representing a Matrix or a Vector.  
    
    """
    
    _ROWS = 0
    _COLS = 1
    
    def __init__ (self, element_data):
        """Creates a new instance of a Tensor.
        
        Args:
            element_data[[row_data]+]    A list/tuple containing other 1 or more lists/tuples representing each row of the Tensor.  Each row must be the same length.
            
        Return:
            Tensor  Instance of the Tensor class, or None if the element_data is not valid.
            
        Raises:
            TypeError        If no parameters are provided or the list is empty
            ValueError       If the elements are not all numberic, or the rows are not the same length
        """
               
        # If element data is (neither a list or tuple) or is empty return an error
        if not (isinstance(element_data, list) or isinstance(element_data, tuple)) or not element_data:
            raise TypeError("Invalid/no parameters .")
        
        # Then make sure the list does actually contain some other non-empty lists
        elif not all(element_data):
            raise ValueError("Element data is incomplete.  Some rows are empty.")
        
        # Check that the rows are the same length and  elements are actually numbers
        else:
            for row in element_data:
                if not (isinstance(row, list) or isinstance(row, tuple)):
                    raise TypeError(f"Row is not a valid input type: {row}")
                
                row_length = len(element_data[0])
                if len(row) != row_length:
                    raise ValueError("Rows are not a uniform length")
                
                if not all(isinstance(value, (int,float)) for value in row):
                    raise ValueError(f"Element data contains non-numeric data in row {row}")
                   
                     
        # By definition a matrix has a fixed number of rows and columns.  
        # The order/dimensions is the number of rows and all rows will contain the same number of columns as the first row
        self.shape = (len(element_data), len(element_data[0]))
        self.elements = tuple(tuple(row) for row in element_data)
        
        
    def __str__(self):
        tensor_str = ''
        for row in self.elements:           
            row_str = ''
            for col_val in row:
                row_str = row_str + ' ' + str(col_val) + ' '            
            tensor_str = tensor_str + row_str + '\n'
    
        return tensor_str
    
    
    #####################################################################################
    # Following methods added (with modifications) from Andrew Kenny's Etivity3
    #####################################################################################
    
    def get_size(self):
        return self.shape
    

    # Ensures matrices have same dimensions
    def valid_for_calculation(self, matrix_2):
        
        if not isinstance(matrix_2, self.__class__):
            return False
        if self.shape != matrix_2.get_size():
            return False
        else:
            return True


    # Function to add or subtract matrices
    def common_calculation(self, tensor_2, lfunc):
        matrix_1 = self.elements
        matrix_2 = tensor_2.elements
        
        result = []


        for i in range(len(matrix_1)):                              #iterate through rows
            row = []
            for j in range(len(matrix_1[0])):                       #iterate through columns
                row.append(lfunc(matrix_1[i][j], matrix_2[i][j]))   #operate on each row
            result.append(row)
        
        return result

    # Adds two matrices

    def add_matrices(self, matrix_2):
        if not self.valid_for_calculation(matrix_2):
            raise ValueError("The provided matrices are not valid for calculation")

        return Tensor(self.common_calculation(matrix_2, lambda x,y: x + y))

    # Subtracts two matrices

    def sub_matrices(self, matrix_2):
        if not self.valid_for_calculation(matrix_2):
            raise ValueError("The provided matrices are not valid for calculation")

        return Tensor(self.common_calculation(matrix_2, lambda x,y: x - y))

    # Multiplies two matrices
    def multiply_matrices(self, tensor_2):
        matrix_1 = self.elements
        matrix_2 = tensor_2.elements
            
        I = range(len(matrix_1))                                     #find rows of matrix_1
        J = range(len(matrix_2[0]))                                  #find columns of matrix_2
        K = range(len(matrix_1[0]))                                  #find columns of matrix_1

        #multiply and add corresponding rows of matrix_1 by columns in matrix_2 (i.e. row 1 by column 1 etc)
        return Tensor([[sum([matrix_1[i][k]*matrix_2[k][j] for k in K]) for j in J] for i in I]) 
    
    
    #####################################################################################
    # Following methods added (with modifications) from my own Etivity3
    #####################################################################################
    def _is_valid_multiplier(self, other):
        """Internal helper function that checks if this tensor and other can be validly tensor.  
        
        To be valid the number of columns in this tensor must equal the number of rows in other.
        """
        
        if not isinstance(other, self.__class__):
            return False
                
        if self.shape[Tensor._COLS] != other.shape[Tensor._ROWS]:
            return False

        return True
    
    
    def _scalar_multiplication(self, other):       
        """Internal method to multiply this matrix by a scalar value."""
        if isinstance(other, (int, float)):
            # Scalar multiplication uses list comprehension with a map to multiply each element by the scalar
            new_elements = [list(map(lambda element: element*other, row)) for row in self.elements]
            new_matrix = Tensor(new_elements)
            return new_matrix
        else:
            return NotImplemented
    
    
    def __eq__(self, other):
        """Compares this tensor object to another tensor object.  Returns true if the shape of the tensors is the same and the elements at each position are the same."""
        
        if self.valid_for_calculation(other):
            for self_row, other_row in zip(self.elements, other.elements):
                if self_row != other_row:
                    return False
            return True           
        else:
            return False
        
        
    def __add__(self, other):
        """Operator overload method for the + operator.  For addition the other tensor must have the same shape as this tensor."""
        return self.add_matrices(other)
        
    def __sub__(self, other):        
        """Operator overload method for the - operator.  For subtraction the other tensor must have the same shape as this tensor."""
        return self.sub_matrices(other)
    
    def __mul__(self, other):

        """Operator overload for the * operator.  Multiplies this matrix by a scalar value.  
        
        Scalar multiplication of a matrix is commutative.  This method handles the case of <matrix> * <scalar>.
        """
        return self._scalar_multiplication(other)

    
    def __rmul__ (self, other):
        
        """Operator overload for the * operator.  Multiplies this matrix by a scalar value.  
        
        Scalar multiplication of a matrix is commutative.  This method handles the case of <scalar> * <matrix>.
        """   
        return self._scalar_multiplication(other)
        
    def __matmul__(self, other):
        """Operator overload for the @ operator.  Multiplies two tensors to return the tensor product.
        
        For multiplication of rank 2 tensors, the number of rows in other must match the number of columns in this tensor.
        """
       
        # Make sure this is a valid multiplication
        if not self._is_valid_multiplier(other):
            return NotImplemented 
        
        return self.multiply_matrices(other)
    
    
    #####################################################################################
    # New methods for Etivity4
    #####################################################################################
    
    def _is_square(self):
        if self.shape[Tensor._ROWS] == self.shape[Tensor._COLS]:
            return True
        else:
            return False
    
    
    def det(self):    
        """Calculates the determinant Tensor of this Tensor object.  
        
        Returns:
            determinant(int, float)    The determinant of the Tensor object as a number
            
        Raises:
            ValueError                 If the Tensor objects does not represent a square Tensor
        """
        
        # Validation check
        if not self._is_square():
            raise ValueError("Determinants are only implemented for square Tensors")
        
        
        # A 1x1 matrix is possible, and the inverse is the element itself
        if self.shape == (1,1):
            return self.elements[0][0]
        
        # A 2x2 matrix is simple
        if self.shape == (2,2):                
            a, b = self.elements[0]
            c, d = self.elements[1]
        
            return ((a*d) - (b*c))  
        else:
            # For larger matrices, the approach is to use the Laplace expansion along the first column
            # This calculates the determinant of the submatrix (called the minor) formed by removing the first row of this matrix, and a specific column
            # Then multiply the minor by the value of first_row[column] and add/subtract from the minor in an alternating pattern 
            determinant = 0
            n_determinant = 0
            
            for col_idx in range(self.shape[Tensor._COLS]):
                
                # Get the submatrix by slicing off the top row and popping out the column values for the column were working over
                submatrix = [list(row) for row in self.elements[1:]]
                for row in submatrix:
                    row.pop(col_idx)

                # Recursive call to get the determinant.  By raising -1 to the power of col_idx we get the alternating +/- without having to use an if statement
                minor = Tensor(submatrix).det()                                                     
                determinant += (-1)**col_idx * self.elements[0][col_idx] * minor                
                
            return determinant
                    
    
    def inverse(self):
        
        """Calculates the inverse Tensor of this Tensor object. Only implemented for 2x2 Tensors 
        
        Returns:
            Tensor       Tensor object representing the inverse of this Tensor
            
        Raises:
            ValueError   If this Tensor object has a determinant = 0, or the Tensor is not a supported size
        """
        
        # A tensor with a determinant of 0 does not have an inverse
        try:
            determinant = self.det()
        except ValueError:
            raise ValueError("Unable to calculate determinant for this Tensor.  Unable to proceed with inverse calculation.")

        if determinant == 0:
            raise ValueError("This tensor has no inverse.  The determinant is 0.")
        elif self.shape != (2,2):
            raise ValueError("This implementation only supports 2x2 Tensor objects.")
        
                
        # For this initial pass we only check for 2x2 matrices (handled here by the call to self.det())
        a, b = self.elements[0]
        c, d = self.elements[1]
        
        # An alternative here would be to simply load the new tensor with elements divided by the determinant
        # This approach is just a little more explicit so its easier to red, which ties in with PEP-20
        reordered_tensor  = Tensor([(d, -b), (-c, a)])      
                
        return (1/determinant) * reordered_tensor
    
    
    def cross_product(self, other):

        """Calculates the cross product of this Tensor object and another Tensor object.  Both objects must represent either a 3x1 or 1x3 vector.
        
        Args:
            other        A Tensor object describing a 3D Tensor
            
        Returns:
            Tensor       Tensor object representing the cross product as a 3x1 vector
            
        Raises:
            TypeError    If the supplied parameter is not another Tensor object
            ValueError   If either of the Tensor objects do not represent a 3 dimensional vector
        """

        # Inputs should be vectors, so other should represent a vector
        if not isinstance(other, self.__class__):
            raise TypeError("Cross Product is only defined in terms of two vectors")       
        elif self.shape != (3, 1) and self.shape != (1, 3):
            raise ValueError("This Tensor object does not represent a 3D Vector.")
        elif other.shape != (3, 1) and other.shape != (1, 3):
            raise ValueError("The parameter Tensor object does not represent a 3D Vector")
            
        # This allows vectors to be represented as either m x 1 or 1 x m
        if self.shape == (3,1):
            v = [row[0] for row in self.elements]
        else:
            v = self.elements[0]
            
        if other.shape == (3,1):
            w = [row[0] for row in other.elements]
        else:
            w = other.elements[0]
        
        # Creates a tuple of tuples.  List notation would also work, but has reduced readability due to element access
        vector_p = ( (v[1]*w[2] - v[2]*w[1],), (v[2]*w[0] - v[0]*w[2],), (v[0]*w[1] - v[1]*w[0],) )
        
        return Tensor(vector_p)
        


### Tests

#### 4 x 4 Matrix Tests

The following 4x4 Matrices, and a 4x1 Vector are defined using the Tensor class for testing:

$
\quad\quad\quad
D = \begin{bmatrix}
    2 & 4 & 7 & 1 \\
    6 & 2 & 4 & 9 \\
    3 & 2 & 6 & 3 \\
    8 & 4 & 5 & 2
\end{bmatrix}
\quad\quad\quad
$
$
\quad\quad\quad
E = \begin{bmatrix}
    8 & 1 & 7 & 4 \\
    2 & 4 & 7 & 1 \\
    4 & 8 & 3 & 2 \\
    1 & 0 & 5 & 9
\end{bmatrix}
\quad\quad\quad
$
$
\quad\quad\quad
\vec{v} = \begin{bmatrix}
    3 \\
    2 \\
    4 \\
    8
\end{bmatrix}
$

The test results using Andrew's code in class form are as follows:

In [2]:
d = Tensor([[2,4,7,1], [6,2,4,9], [3,2,6,3], [8,4,5,2]])
e = Tensor(((8,1,7,4), (2,4,7,1), (4,8,3,2), (1,0,5,9)))
v = Tensor(([3], [2], [4], [8]))

print(f"The size of Tensor D is {d.get_size()}")
print(f"The size of Tensor E is {e.get_size()}")
print(f"The size of Tensor v is {v.get_size()}")

print()

print(f"D + E =\n{d + e}")
print(f"D - E =\n{d - e}")
print(f"D . E =\n{d @ e}")
print(f"D . v =\n{d @ v}")

The size of Tensor D is (4, 4)
The size of Tensor E is (4, 4)
The size of Tensor v is (4, 1)

D + E =
 10  5  14  5 
 8  6  11  10 
 7  10  9  5 
 9  4  10  11 

D - E =
 -6  3  0  -3 
 4  -2  -3  8 
 -1  -6  3  1 
 7  4  0  -7 

D . E =
 53  74  68  35 
 77  46  113  115 
 55  59  68  53 
 94  64  109  64 

D . v =
 50 
 110 
 61 
 68 



### New functionality tests

For the new _determinant_, _inverse_ and _cross product_ operations, the following 2x2 matrices are used:

$
\quad\quad\quad
A = \begin{bmatrix}
    1 & 3 \\
    5 & 7
\end{bmatrix}
\quad\quad\quad
$
$
\quad\quad\quad
B = \begin{bmatrix}
    3 & 2 \\
    4 & 6
\end{bmatrix}
\quad\quad\quad
$
$
\quad\quad\quad
C = \begin{bmatrix}
    2 & 0 \\
    0 & 4
\end{bmatrix}
\quad\quad\quad
$

#### Determinant

In [3]:
a = Tensor(((1,3), (5,7)))
b = Tensor(((3,2), (4,6)))        
c = Tensor([[2,0], [0,4]])

print(f"|A| = {a.det()}\n")
print(f"|B| = {b.det()}\n")
print(f"|C| = {c.det()}\n")

|A| = -8

|B| = 10

|C| = 8



_Reflection Addition_

Following a rewrite of my determinant method it's no longer restricted to 2x2 matrices.  Using the larger matrices defined above, and a new one:

$$
\begin{align}
\det \begin{bmatrix}
    1 & 2 & 3 \\ 0 & 3 & 4 \\ 5 & 6 & 8\end{bmatrix}
&= 1 \cdot \det \begin{bmatrix} 3 & 4 \\6 & 8 \end{bmatrix}
- 2 \cdot \det \begin{bmatrix} 0 & 4 \\5 & 8 \end{bmatrix} 
+ 3 \cdot \det \begin{bmatrix} 0 & 3 \\5 & 6 \end{bmatrix} \\
&= 1(3\cdot8 - 4 \cdot 6) - 2(0\cdot8 - 4\cdot5) +3 (0\cdot6 - 3\cdot5) \\
&= 1(0) - 2(-20) + 3(-15) \\
&= 0 + 40 - 45 \\
&= -5
\end{align}
$$


In [4]:
print(f"The determinant of the 3x3 Tensor above is: {Tensor([[1,2,3], [0,3,4], [5,6,8]]).det()}")

The determinant of the 3x3 Tensor above is: -5


While not the most computationally efficient approach, the method can scale higher.  At this scale the manual calculations are quite long to write out, so I've validated using calculators at [Matrix Calculator](https://matrixcalc.org) and [Symbolab](https://www.symbolab.com/solver/matrix-determinant-calculator).

 Using the 4x4 matrices from earlier:

In [5]:
print(f"|D| = {d.det()}\n")
print(f"|E| = {e.det()}\n")

|D| = 470

|E| = -2805



#### Inverse

Next up, the inverse of the tensors:

In [6]:
print(f"Inverse of A = \n{a.inverse()}")
print(f"Inverse of B = \n{b.inverse()}")
print(f"Inverse of C = \n{c.inverse()}")

Inverse of A = 
 -0.875  0.375 
 0.625  -0.125 

Inverse of B = 
 0.6000000000000001  -0.2 
 -0.4  0.30000000000000004 

Inverse of C = 
 0.5  0.0 
 0.0  0.25 



_Reflection Addition_

While the functionality isn't changed from my original posting on October 6th, the implementation has changed.  I considered an approach using a new list of re-ordered elements, combined with a list comprehension function.  I suggested this in my feedback to Cathal Cronin.

```python
        new_order = [(self.elements[1][1], -self.elements[0][1]), (-self.elements[1][0], self.elements[0][0])]
             
        return Tensor([ [val/determinant for val in row] for row in new_order])
```

In the end I opted to re-introduce the internal `_scalar_multiplication` method I had in my Etivity 3 class.  Using this, I then created a reordered Tensor object and multiplied that by the inverse of the determinant.  This was a very literal implementation of the formula for inverse of a 2x2 matrix.

My work on updating the `det` method gave me an initial outline for approaching a general nxn case using a matrix of minors, transposition and the determinant.  In the end time doesn't allow to follow that path. 

### Cross Product

For the cross product tests, I use the following tensors

$
\quad\quad\quad
\vec{h} = \begin{bmatrix}
    2 \\
    1 \\
    3
\end{bmatrix}
\quad\quad\quad
$
$
\quad\quad\quad
\vec{l} = \begin{bmatrix}
    3 \\
    4 \\
    4
\end{bmatrix}
\quad\quad\quad
$

_Reflection Modification_

My initial posting from October 6th was incorrect and calculated a scalar product for a 2x2 matrix.  At the time I hadn't understood that it was to be a vector product, so this method was modified.  While my initial failure was to go through the full material on cross product, I did spend a lot of time reading up on the idea in general, and trying to understand the geometry behind it.

See above under _Approach_ for my reasoning on confining the method to explicitly 3D vectors. 

In [7]:
h = Tensor([[2], [1], [3]])
l = Tensor([[3], [4], [4]])

print(f"The cross product h x l is\n{h.cross_product(l)}")
print(f"The cross product l x h is\n{l.cross_product(h)}")


The cross product h x l is
 -8 
 1 
 5 

The cross product l x h is
 8 
 -1 
 -5 



The code also allows for them to be written as:

$
\quad\quad\quad
\vec{h} = \begin{bmatrix}
    2 & 1 & 3
\end{bmatrix}
\quad\quad\quad
$
$
\quad\quad\quad
\vec{l} = \begin{bmatrix}
    3 & 4 & 4
\end{bmatrix}
\quad\quad\quad
$

In [8]:
h = Tensor([[2,1,3]])
j = Tensor([[3,4,4]])

print(f"The cross product h x j is\n{h.cross_product(j)}")
print(f"The cross product j x h is\n{j.cross_product(h)}")

The cross product h x j is
 -8 
 1 
 5 

The cross product j x h is
 8 
 -1 
 -5 



It's even possible to mix the two, but I don't know if that is valid.

___
### Unit Tests

The unittests were mostly derived from my previous Etivity 3 unit tests.  They formally structure many of the calculations above.  New tests for the `det`, `inverse` and `cross_product` methods are included.

_Reflection Addition_

Based on a suggestion from **Brian Parle** (posted to Vipul Polpat) I updated the inverse check to validate that not only were the return values what I expected, but that the Tensors and their inverses when multiplied yielded the Identity Matrix.

In [9]:
# Student Name:    Mark Murnane
# Student ID:      18195326

import unittest

class TestTensorImplementationEtivity4(unittest.TestCase):
    
    
    """Tests the implementation of the Tensor class for Etivity 4.
    
    Unit tests are:
        1) Create a new tensor and confirm size is correctly reported in a tuple
        2) Add two matrices
        3) Subtract two matrices
        5) Multiply a tensor by a scalar
        6) Multiply a tensor by a vector
        7) Multiply a tensor by a tensor
        8) Test equality and inequality
        9) Calculate the determinant of a 2 x 2 Tensor
        
    Edge Cases in this Unit Test are:
        1) Create a new tensor with an empty list or non-list parameter
        2) Create a tensor with non-list rows
        3) Create a tensor with empty rows
        4) Create a tensor with rows that are not the same length
        5) Perform addition/subtraction of a tensor and a non-tensor
        6) Perform addition/subtraction of a tensor and an incompatible tensor
        7) Multiply a tensor by a non-numeric value
        8) Multiply a tensor by an incompatible tensor
    """
    
    # Internal tensor and (column) vector representations for test cases
    def setUp(self):
        self._A_2x2 = ((1,3), (5,7))
        self._B_2x2 = ((2,4), (6,8))
        
        self._Err_A_2x2 = [[1,3], ["5", 7]]
        self._Err_B_2x2 = [[2,4], []]
        self._Err_C_2x2 = [[1,2], [4,5,6]]
        self._Err_D = [[3, 4], 5]
        
        self._B_2x1 = [[2], [1]]
        
        self._A_3x3 = [[1,2,3], [0,3,4], [5,6,8]]
        self._B_3x3 = [[1,3,6], [2,1,2], [4,2,1]]
        
        self._B_3x1 = [[2], [1], [3]]
        
        self._B_3x2 = [[4,7], [3,9], [5,2]]
    
        
    def test_tensor_creation(self):
        # Checks that a tensor is created successfully with the correct dimensions and content
        a = Tensor(self._A_2x2)
        self.assertEqual(a.get_size(), (2,2))
        self.assertEqual(a.elements, self._A_2x2)
    
        
    def test_tensor_invalid_creation(self):
        # Checks that a tensor can't be created without a valid element list
        self.assertRaises(TypeError, Tensor, None)
        self.assertRaises(TypeError, Tensor, [])
        self.assertRaises(TypeError, Tensor, self._Err_D)
        self.assertRaises(ValueError, Tensor, self._Err_A_2x2)
        self.assertRaises(ValueError, Tensor, self._Err_B_2x2)
        self.assertRaises(ValueError, Tensor, self._Err_C_2x2)
        
    
        
    def test_tensor_add_matrices(self):
        # Checks that two same size matrices are added together correctly
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = a + b
        expected = ((3,7), (11,15))
        self.assertEqual(c.elements, expected)
    
        
    def test_tensor_add_invalid_matrices(self):
        # Checks that two same differing size matrices can't be added or subtracted
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_3x3)
        
        func = lambda x, y: x + y
        
        self.assertRaises(ValueError, func, a, b)
        self.assertRaises(ValueError, func, a, 3)
        self.assertRaises(ValueError, func, a, "")
               
        
    def test_tensor_subtract_matrices(self):
        # Checks that two same size matrices are added together correctly
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = a - b
        expected = ((-1,-1), (-1,-1))
        self.assertEqual(c.elements, expected)
    
        
    def test_tensor_subtract_invalid_matrices(self):
        # Checks that two same differing size matrices can't be added or subtracted
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_3x3)
        
        func = lambda x, y: x - y
        
        self.assertRaises(ValueError, func, a, b)
        self.assertRaises(ValueError, func, a, 3)
        self.assertRaises(ValueError, func, a, "")


    def test_tensor_multiply_by_scalar(self):
        # Checks the correct multiplication of a tensor by a scalar      
        a = Tensor(self._B_3x2)
        b = a * 3
        expected = ((12,21), (9,27), (15,6))
        self.assertEqual(b.elements, expected)


    def test_tensor_multiply_by_vector(self):
        # Checks the correct multiplication of a tensor by a vector      
        a = Tensor(self._B_2x2)
        b = Tensor(self._B_2x1)
        c = a @ b
        expected = ((8,),(20,))
        self.assertEqual(c.elements, expected)

        
    def test_tensor_multiply_same_order_matrices(self):
        # Checks that two same size matrices are multiplied together correctly
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)
        c = a @ b
        expected = ((20,28), (52,76))
        self.assertEqual(c.elements, expected)
        
        
    def test_tensor_multiply_nonsquare_matrices(self):
        # Checks that two different size matrices are multiplied together correctly
        a = Tensor(self._A_3x3)
        b = Tensor(self._B_3x2)
        c = a @ b
        expected = ((25,31), (29,35), (78,105))
        self.assertEqual(c.elements, expected)
        
        
    def test_tensor_multiply_incompatible_matrices(self):
        # Checks that matrices with incompatible dimension/order can't be multiplied
        a = Tensor(self._A_3x3)
        b = Tensor(self._B_3x2)
        
        func = lambda x, y: x * y
        
        self.assertRaises(TypeError, func, b, a)
        
        
    def test_tensor_multiply_non_numeric(self):
        # Checks that matrices can't be multiplied by a string or non-Tensor object
        a = Tensor(self._A_2x2)
        
        func = lambda x, y: x * y
        
        self.assertRaises(TypeError, func, a, "")
        self.assertRaises(TypeError, func, a, None)
        self.assertRaises(TypeError, func, a, [])
        
    def test_tensor_equality(self):
        # Check that tensor equality rules are correct
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_2x2)        
        c = Tensor([[1,3], [5,7]])
        d = Tensor(self._A_3x3)
        
        self.assertTrue(a != b)
        self.assertTrue(a == c)
        self.assertFalse(a == d)
        self.assertFalse(c == d)
        
        
    def test_tensor_determinant(self):
        a = Tensor(self._A_2x2)
        b = Tensor(self._B_3x3)        
        c = Tensor(((8,1,7,4), (2,4,7,1), (4,8,3,2), (1,0,5,9)))
        
        # Check a non-square matrix will fail
        with self.assertRaises(ValueError):
            Tensor(self._B_2x1).det()
        
        self.assertEqual(a.det(), -8)
        self.assertEqual(b.det(), 15)
        self.assertEqual(c.det(), -2805)
        
        
    def test_tensor_inverse(self):
        a = Tensor(self._A_2x2)      
        c = Tensor([[2,0], [0,4]])
        
        identity = Tensor([[1,0], [0,1]])
        
        a_inverse = a.inverse()
        c_inverse = c.inverse()
        
        expected_a = ((-0.875, 0.375), (0.625, -0.125))
        expected_c = ((0.5, 0), (0, 0.25))
        
        self.assertEqual(a_inverse.elements, expected_a)
        self.assertEqual(c_inverse.elements, expected_c)
                    
        self.assertEqual(a @ a_inverse, identity)
        self.assertEqual(c @ c_inverse, identity)
        
        
    def test_tensor_cross_product(self):
        h = Tensor([[2], [1], [3]])
        j = Tensor([[3], [4], [4]])

        h_x_j = h.cross_product(j)
        j_x_h = j.cross_product(h)

        expected_h_x_j = ((-8,), (1,), (5,)) 
        expected_j_x_h = ((8,), (-1,), (-5,))
        
        self.assertEqual(h_x_j.elements, expected_h_x_j)
        self.assertEqual(j_x_h.elements, expected_j_x_h)
            

            
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False, verbosity=2)

test_tensor_add_invalid_matrices (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_add_matrices (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_creation (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_cross_product (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_determinant (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_equality (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_invalid_creation (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_inverse (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_multiply_by_scalar (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_multiply_by_vector (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_multiply_incompatible_matrices (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_multiply_non_numeric (__main__.TestTensorImplementationEtivity4) ... ok
test_tensor_multiply_nonsquare_matrices (__main__.T